# Logistic Regression for Sentiment Analysis

Adapted from http://nbviewer.jupyter.org/github/rasbt/pattern_classification/blob/master/machine_learning/scikit-learn/outofcore_modelpersistence.ipynb

<br>
<br>

## The IMDb Movie Review Dataset

In this section, we will train a simple logistic regression model to classify movie reviews from the 50k IMDb review dataset that has been collected by Maas et. al.

> AL Maas, RE Daly, PT Pham, D Huang, AY Ng, and C Potts. Learning word vectors for sentiment analysis. In Proceedings of the 49th Annual Meeting of the Association for Computational Lin- guistics: Human Language Technologies, pages 142–150, Portland, Oregon, USA, June 2011. Association for Computational Linguistics

[Source: http://ai.stanford.edu/~amaas/data/sentiment/]

The dataset consists of 50,000 movie reviews from the original "train" and "test" subdirectories. The class labels are binary (1=positive and 0=negative) and contain 25,000 positive and 25,000 negative movie reviews, respectively.
For simplicity, I assembled the reviews in a single CSV file.


In [3]:
import pandas as pd
# if you want to download the original file:
#df = pd.read_csv('https://raw.githubusercontent.com/rasbt/pattern_classification/master/data/50k_imdb_movie_reviews.csv')
# otherwise load local file
df = pd.read_csv('shuffled_movie_data.csv')
df.tail()

,review,sentiment
49995,Never viewed this film until recently on TCM a...,1
49996,"""Return of the Seven"" has a few good action sc...",0
49997,I watched the first 10 minutes of this show I ...,0
49998,Dragon Hunters has to be the best-looking anim...,1
49999,This movie is about two guys who made up a spo...,1


In [4]:
positive = pd.read_csv('positive-words.txt')
positive.tail()

,words
2002,youthful
2003,zeal
2004,zenith
2005,zest
2006,zippy


In [5]:
negative = pd.read_csv('negative-words.txt', encoding = "ISO-8859-1")
negative.tail()

,words
4776,zaps
4777,zealot
4778,zealous
4779,zealously
4780,zombie


Let us shuffle the class labels.

In [6]:
import numpy as np
## uncomment these lines if you have dowloaded the original file:
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df[['review', 'sentiment']].to_csv('shuffled_movie_data.csv', index=False)
df.tail()

,review,sentiment
21243,This film was reeeeeeallyyyy bad! Was it meant...,0
45891,Meek and mild Edward G. Robinson (as Wilbert W...,0
42613,"I guess I have to write something here, althou...",1
43567,A while back I bought the Chinese box set of F...,1
2732,"When watching little man , you'll spend its ru...",0


<br>
<br>

## Preprocessing Text Data

Now, let us define a simple `tokenizer` that splits the text into individual word tokens. Furthermore, we will use some simple regular expression to remove HTML markup and all non-letter characters but "emoticons," convert the text to lower case, remove stopwords, and apply the Porter stemming algorithm to convert the words into their root form.

In [12]:
import numpy as np
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
print(stop)
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    text = [w for w in text.split() if w not in stop]
    tokenized = [porter.stem(w) for w in text]
    return text

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Let's give it at try:

In [ ]:
tokenizer('This :) is a <a> test! :-)</br>')

## Learning (SciKit)

First, we define a generator that returns the document body and the corresponding class label:

In [8]:
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

To conform that the `stream_docs` function fetches the documents as intended, let us execute the following code snippet before we implement the `get_minibatch` function:

In [ ]:
next(stream_docs(path='shuffled_movie_data.csv'))

After we confirmed that our `stream_docs` functions works, we will now implement a `get_minibatch` function to fetch a specified number (`size`) of documents:

In [9]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    for _ in range(size):
        text, label = next(doc_stream)
        docs.append(text)
        y.append(label)
    return docs, y

Next, we will make use of the "hashing trick" through scikit-learns [HashingVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html) to create a bag-of-words model of our documents. Details of the bag-of-words model for document classification can be found at  [Naive Bayes and Text Classification I - Introduction and Theory](http://arxiv.org/abs/1410.5329).

In [ ]:
# from sklearn.feature_extraction.text import HashingVectorizer
# vect = HashingVectorizer(decode_error='ignore', 
#                          n_features=2**21,
#                          preprocessor=None, 
#                          tokenizer=tokenizer)

# Excercise 1: define new features according to https://web.stanford.edu/~jurafsky/slp3/5.pdf

def getFeatures(X_test, positve, negative, tokenizer):
    Xfeatured = pd.DataFrame(columns=['x1','x2','x3','x4','x5','x6'])
    prons = ['i', 'me', 'my', 'mine', 'we', 'us', 'our', 'ours', 'you', 'your',  'yours']
    
    for review in X_test:
        wlist = tokenizer(review)
        x1 = 0 # count positive words
        x2 = 0 # count negative words
        x3 = 0 # 'no' in doc
        x4 = 0 # count pronouns
        x5 = 0 # '!' in doc
        x6 = 0 # log(count of words)

        for w in wlist:
            if positive['words'].str.contains(pat=w, case=False, regex=False).any():
                x1+=1
            elif negative['words'].str.contains(pat=w, case=False, regex=False).any():
                x2+=1
            elif w in ['no']:
                x3 = 1
            elif w in prons:
                x4+=1
            elif w in ['!']:
                x5 = 1

        x6 = np.log(len(wlist))
#         for i in range(len(positive)):
#             counter = df['review'].str.count(positive['word'][i])
#             df['x1'] = counter + df['x1']

#         for i in range(len(negative)):
#             counter = df['review'].str.count(negative['word'][i])
#             df['x2'] = counter + df['x2']

#         df['x3'] = df['review'].str.contains('no', case=False).astype(int)
#         df['x4'] = df['review'].str.count('i ') + \
#             df['review'].str.count('I ')   + \
#             df['review'].str.count('I\'')  + \
#             df['review'].str.count('i\'')  + \
#             df['review'].str.count('you ') + \
#             df['review'].str.count('You ') + \
#             df['review'].str.count(' u ')  + \
#             df['review'].str.count(' U ')  + \
#             df['review'].str.count(' u\'')

#         df['x5'] = df['review'].str.contains('!', case=False).astype(int)
#         df['x6'] = np.log(df['review'].str.count(' ').add(1))

        Xfeatured = Xfeatured.append({
         'x1': x1,
         'x2': x2,
         'x3': x3,
         'x4': x4,
         'x5': x5,
         'x6': x6,
        }, ignore_index=True)
    
    return Xfeatured

doc_stream = stream_docs(path='shuffled_movie_data.csv')
X_train, Y_train = get_minibatch(doc_stream, size=1000)
X_train = getFeatures(X_train, positive, negative, tokenizer)
print (Y_train)


Using the [SGDClassifier]() from scikit-learn, we will can instanciate a logistic regression classifier that learns from the documents incrementally using stochastic gradient descent. 

In [ ]:
#from sklearn.linear_model import SGDClassifier
#clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
#doc_stream = stream_docs(path='shuffled_movie_data.csv')

# Excercise 2: implement a Logistic Regression classifier, using regularization, according to https://web.stanford.edu/~jurafsky/slp3/5.pdf

def sigmoid(x):
    return 1.0 / (1 + np.exp(-x))

def stocashticGradientDescent(X,Y,theta,alpha=0.01,iterations=10):
    m = len(Y)
    
    for it in range(iterations):
        X_i = X.values
        Y_i = Y
        prediction = sigmoid(np.dot(X_i,theta))
        g = np.dot(prediction-Y_i,X_i)
        theta = theta - (1/m)*alpha*g
        
    return theta



In [ ]:
#import pyprind
#pbar = pyprind.ProgBar(45)

# classes = np.array([0, 1])

# for _ in range(45):
#     X_train, Y_train = get_minibatch(doc_stream, size=1000)
#     X_train = vect.transform(X_train)
#     type(X_train)
#     clf.partial_fit(X_train, Y_train, classes=classes)
#     #pbar.update()

theta = np.random.rand(6)
for i in range(45):
#     X_train, Y_train = get_minibatch(doc_stream, size=1000)
#     X_train = getFeatures(X_train, positive, negative, tokenizer)

    X_train_bath = X_train.loc[(X_train['id'] >= i*1000) & (X_train['id'] < (i+1)*1000), ['x1', 'x2', 'x3', 'x4', 'x5', 'x6']] 
    Y_train_batch = Y_train[]
    theta = stocashticGradientDescent(X_train,Y_train,theta,alpha=0.01,iterations=1)
    print (theta)

Depending on your machine, it will take about 2-3 minutes to stream the documents and learn the weights for the logistic regression model to classify "new" movie reviews. Executing the preceding code, we used the first 45,000 movie reviews to train the classifier, which means that we have 5,000 reviews left for testing:

In [ ]:
X_test, Y_test = get_minibatch(doc_stream, size=5000)
# X_test = vect.transform(X_test)
# print('Accuracy: %.3f' % clf.score(X_test, y_test))

X_test = getFeatures(X_train, positive, negative, tokenizer)
predictons = sigmoid(np.dot(X_i,theta))

from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, predictons)

I think that the predictive performance, an accuracy of ~87%, is quite "reasonable" given that we "only" used the default parameters and didn't do any hyperparameter optimization. 

After we estimated the model perfomance, let us use those last 5,000 test samples to update our model.

In [ ]:
# clf = clf.partial_fit(X_test, y_test)

<br>
<br>

# Model Persistence

In the previous section, we successfully trained a model to predict the sentiment of a movie review. Unfortunately, if we'd close this IPython notebook at this point, we'd have to go through the whole learning process again and again if we'd want to make a prediction on "new data."

So, to reuse this model, we could use the [`pickle`](https://docs.python.org/3.5/library/pickle.html) module to "serialize a Python object structure". Or even better, we could use the [`joblib`](https://pypi.python.org/pypi/joblib) library, which handles large NumPy arrays more efficiently.

To install:
conda install -c anaconda joblib

In [ ]:
import joblib
import os
if not os.path.exists('./pkl_objects'):
    os.mkdir('./pkl_objects')
    
joblib.dump(vect, './vectorizer.pkl')
joblib.dump(clf, './clf.pkl')

Using the code above, we "pickled" the `HashingVectorizer` and the `SGDClassifier` so that we can re-use those objects later. However, `pickle` and `joblib` have a known issue with `pickling` objects or functions from a `__main__` block and we'd get an `AttributeError: Can't get attribute [x] on <module '__main__'>` if we'd unpickle it later. Thus, to pickle the `tokenizer` function, we can write it to a file and import it to get the `namespace` "right".

In [ ]:
%%writefile tokenizer.py
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    text = [w for w in text.split() if w not in stop]
    tokenized = [porter.stem(w) for w in text]
    return text

In [ ]:
from tokenizer import tokenizer
joblib.dump(tokenizer, './tokenizer.pkl')

Now, let us restart this IPython notebook and check if the we can load our serialized objects:

In [ ]:
import joblib
tokenizer = joblib.load('./tokenizer.pkl')
vect = joblib.load('./vectorizer.pkl')
clf = joblib.load('./clf.pkl')

After loading the `tokenizer`, `HashingVectorizer`, and the tranined logistic regression model, we can use it to make predictions on new data, which can be useful, for example, if we'd want to embed our classifier into a web application -- a topic for another IPython notebook.

In [ ]:
example = ['I did not like this movie']
X = vect.transform(example)
clf.predict(X)

In [ ]:
example = ['I loved this movie']
X = vect.transform(example)
clf.predict(X)